In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

In [2]:
import torch
torch.cuda.is_available()

True

- 데이터 전처리

In [3]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),              # tensor 변환
        transforms.Resize(32),              # 이미지 크기변경
        transforms.Normalize((0.5), (1.0))  # 정규화 (평균, 표준편차)
    ]
)

train_data = datasets.MNIST(root="./data/", train=True, download=True, transform=data_transform)
test_data = datasets.MNIST(root="./data/", train=False, download=True, transform=data_transform)

100%|██████████| 9.91M/9.91M [00:06<00:00, 1.43MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 164kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.14MB/s]


In [7]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [10]:
data, label = next(iter(train_loader))
print(data.shape)

torch.Size([32, 1, 32, 32])


In [11]:
class Lenet(nn.Module) :     # class 이름이 곧 모델명(Lenet)
    def __init__(self) :
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x) :
        x = self.conv1(x)
        x = F.tanh(x)               # 6, 28, 28 (활성화함수)
        x = F.max_pool2d(x, 2, 2)   # 6, 14, 14 (2X2 max pooling)

        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)

        x = self.fc1(x)
        x = F.tanh(x)

        x = self.fc2(x)
        x = F.tanh(x)

        return x

In [12]:
model = Lenet()
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [13]:
from torchsummary import summary

In [15]:
# 학습시키기
lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 10
criterion = nn.CrossEntropyLoss()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

writer = SummaryWriter()
step = 0

for epoch in range(epochs) :
    for data, label in train_loader :
        optim.zero_grad()
        pred = model(data.to(device))  # [32, 1, 32, 32]
        loss = criterion(pred, label.to(device))
        writer.add_scalar("Loss/train", loss, step)
        step += 1

        loss.backward()
        optim.step()

    print(f"{epoch+1} loss ::: {loss.item()}")

1 loss ::: 0.8765782117843628
2 loss ::: 0.8323983550071716
3 loss ::: 0.7984861135482788
4 loss ::: 0.9186980724334717
5 loss ::: 0.8189929127693176
6 loss ::: 0.7969552278518677
7 loss ::: 0.8450007438659668
8 loss ::: 0.8107438087463379
9 loss ::: 0.799386739730835
10 loss ::: 0.8078277111053467


- 모델 저장

In [16]:
torch.save(model.state_dict(), "model/mnistModel_weight.pth")